<a href="https://colab.research.google.com/github/snehakap/Car-detection-using-ML/blob/main/Car_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import cv2
from tracker import EuclideanDistTracker
from google.colab.patches import cv2_imshow
import os

# Create tracker object
tracker = EuclideanDistTracker()

cap = cv2.VideoCapture("/content/input_video.mp4")

# Object detection from Stable camera
object_detector = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=100)

# Create a directory to save frames
output_directory = "/content/frames"
os.makedirs(output_directory, exist_ok=True)

frame_count = 0
total_cars_passed = -1
prev_objects = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width, _ = frame.shape

    # Extract Region of interest
    roi = frame[190:352, 270:600]

    # Object Detection
    mask = object_detector.apply(roi)
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    detections = []

    for cnt in contours:
        # Calculate area and remove small elements
        area = cv2.contourArea(cnt)
        if area > 7000:
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 3)
            detections.append([x, y, w, h])

    # Object Tracking
    boxes_ids = tracker.update(detections)
    for box_id in boxes_ids:
        x, y, w, h, id = box_id
        cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 3)
        if y < 10 and id not in prev_objects:  # Check if the object crosses the top boundary
            total_cars_passed += 1
            prev_objects.append(id)

    # Display the count on top of the frame
    cv2.putText(frame, f"Total Cars Passed: {total_cars_passed}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    #cv2_imshow(frame)

    # Save the frame
    frame_count += 1
    frame_filename = os.path.join(output_directory, f"frame_{frame_count:04d}.jpg")
    cv2.imwrite(frame_filename, frame)

    key = cv2.waitKey(30)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()



{0: (172, 65)}
{0: (184, 69)}
{1: (217, 82)}
{1: (219, 85)}
{1: (226, 88)}
{1: (250, 92)}
{1: (255, 97)}
{1: (249, 97)}
{1: (266, 100)}
{2: (99, 43)}
{2: (111, 48)}
{2: (124, 53)}
{2: (133, 59)}
{2: (146, 65)}
{2: (161, 73)}
{2: (177, 81)}
{2: (194, 82)}
{2: (208, 85)}
{2: (213, 91)}
{2: (226, 95)}
{2: (234, 97)}
{2: (244, 103)}
{2: (253, 108)}
{2: (265, 112)}
{2: (278, 116)}
{3: (157, 63)}
{3: (172, 70)}
{3: (185, 78)}
{3: (201, 82)}
{3: (216, 85)}
{3: (229, 88)}
{3: (234, 91)}
{3: (241, 94)}
{3: (250, 100)}
{3: (259, 102)}
{3: (267, 106)}
{4: (99, 44)}
{4: (102, 48)}
{4: (112, 52)}
{4: (121, 57)}
{4: (130, 62)}
{4: (140, 69)}
{4: (153, 75)}
{4: (165, 81)}
{4: (177, 81)}
{4: (191, 81)}
{4: (195, 81)}
{4: (198, 81)}
{4: (203, 81)}
{4: (208, 81)}
{4: (214, 81)}
{4: (221, 81)}
{4: (228, 83)}
{4: (233, 85)}
{4: (245, 88)}
{4: (249, 90)}
{4: (258, 93)}
{4: (269, 97)}
{4: (279, 111)}
{5: (132, 45)}
{5: (141, 49)}
{5: (150, 53)}
{5: (161, 58)}
{5: (172, 63)}
{5: (183, 68)}
{5: (196, 74)}
{5:

In [9]:
#Convert the frames to video format
!ffmpeg -i /content/frames/frame_%04d.jpg -c:v libx264 -vf "fps=30,format=yuv420p" /content/output_video.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab